<a href="https://colab.research.google.com/github/thebhulawat/audioLM/blob/main/audiolm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Apr 17 09:25:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install audiolm-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.7/903.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.

In [ ]:
import math
import wave
import struct
import os
import urllib.request
import tarfile
from audiolm_pytorch import AudioLM, SoundStream, SoundStreamTrainer, HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer, CoarseTransformer, CoarseTransformerWrapper, FineTransformer, FineTransformerWrapper,  CoarseTransformerTrainer, FineTransformerTrainer
from torch import nn
import torch
import torchaudio
dataset_folder = "dataset"
soundstream_ckpt = "results/soundstream.8.pt"
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin'

In [ ]:
# Placeholder data generation
def get_sinewave(freq=440.0, duration_ms=200, volume=1.0, sample_rate=44100.0):
  # code adapted from https://stackoverflow.com/a/33913403
  audio = []
  num_samples = duration_ms * (sample_rate / 1000.0)
  for x in range(int(num_samples)):
    audio.append(volume * math.sin(2 * math.pi * freq * (x / sample_rate)))
  return audio

def save_wav(file_name, audio, sample_rate=44100.0):
  # Open up a wav file
  wav_file=wave.open(file_name,"w")
  # wav params
  nchannels = 1
  sampwidth = 2
  # 44100 is the industry standard sample rate - CD quality.  If you need to
  # save on file size you can adjust it downwards. The stanard for low quality
  # is 8000 or 8kHz.
  nframes = len(audio)
  comptype = "NONE"
  compname = "not compressed"
  wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))
  # WAV files here are using short, 16 bit, signed integers for the
  # sample size.  So we multiply the floating point data we have by 32767, the
  # maximum value for a short integer.  NOTE: It is theortically possible to
  # use the floating point -1.0 to 1.0 data directly in a WAV file but not
  # obvious how to do that using the wave module in python.
  for sample in audio:
      wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
  wav_file.close()
  return

def make_placeholder_dataset():
  # Make a placeholder dataset with a few .wav files that you can "train" on, just to verify things work e2e
  if os.path.isdir(dataset_folder):
    return
  os.makedirs(dataset_folder)
  save_wav(f"{dataset_folder}/example.wav", get_sinewave())
  save_wav(f"{dataset_folder}/example2.wav", get_sinewave(duration_ms=500))
  os.makedirs(f"{dataset_folder}/subdirectory")
  save_wav(f"{dataset_folder}/subdirectory/example.wav", get_sinewave(freq=330.0))

make_placeholder_dataset()


In [ ]:
soundstream = SoundStream (
    codebook_size = 1024,
    rq_num_quantizers = 8
)
trainer = SoundStreamTrainer(
    soundstream,
    folder = dataset_folder,
    batch_size = 1,
    grad_accum_every = 8,
    data_max_length = 320 * 32,
    save_results_every = 2,
    save_model_every = 4,
    num_train_steps = 50
).cuda()

trainer.train()

training with dataset of 56 samples and validating with randomly splitted 3 samples
do you want to clear previous experiment checkpoints and results? (y/n) y
0: soundstream total loss: 22.605, soundstream recon loss: 0.022 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 2.000
0: saving to results
0: saving model to results
1: soundstream total loss: 26.519, soundstream recon loss: 0.030 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 2.000
2: soundstream total loss: 29.534, soundstream recon loss: 0.037 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 2.000
2: saving to results
3: soundstream total loss: 25.056, soundstream recon loss: 0.029 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | discr (scale 0.25) loss: 2.000
4: soundstream total loss: 19.311, soundstream recon loss: 0.021 | discr (scale 1) loss: 2.000 | discr (scale 0.5) loss: 2.000 | disc

In [ ]:
if not os.path.isdir('hubert'):
  os.makedirs('hubert')
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download =  f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 50
)

trainer.train()

training with dataset of 56 samples and validating with randomly splitted 3 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 6.1783528327941895
0: valid loss 6.4643874168396
0: saving model to results
1: loss: 6.646292209625244
2: loss: 6.049803733825684
3: loss: 6.628210544586182
4: loss: 6.395344257354736
5: loss: 5.762567520141602
6: loss: 5.6280436515808105
7: loss: 5.479274272918701
8: loss: 5.779816150665283
9: loss: 5.933629989624023
10: loss: 6.008376121520996
11: loss: 5.710422515869141
12: loss: 5.516768455505371
13: loss: 5.577199935913086
14: loss: 5.763340473175049
15: loss: 5.511303424835205
16: loss: 5.876579761505127
17: loss: 5.782003402709961
18: loss: 5.3636322021484375
19: loss: 4.860294818878174
20: loss: 6.998456001281738
21: loss: 5.625324249267578
22: loss: 5.963174343109131
23: loss: 5.231822490692139
24: loss: 5.325855255126953
25: loss: 5.7134690284729
26: loss: 4.7604475021362305
27: loss: 5.041004657745361
28: loss:

In [ ]:
wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

soundstream.load(f'./{soundstream_ckpt}')

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    codec = soundstream,
    wav2vec = wav2vec,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 *32,
    save_results_every = 2,
    save_model_every = 4,
    num_train_steps = 50
)

trainer.train()

training with dataset of 56 samples and validating with randomly splitted 3 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 49.28752517700195
0: valid loss 56.12900924682617
0: saving model to results
1: loss: 50.54399108886719
2: loss: 67.16716003417969
2: valid loss 58.75642013549805
3: loss: 56.899375915527344
4: loss: 46.982269287109375
4: valid loss 54.066959381103516
4: saving model to results
5: loss: 43.24955749511719
6: loss: 42.151893615722656
6: valid loss 35.54770278930664
7: loss: 42.31438446044922
8: loss: 41.86770248413086
8: valid loss 31.354446411132812
8: saving model to results
9: loss: 17.625350952148438
10: loss: 40.83538818359375
10: valid loss 27.665346145629883
11: loss: 16.7656192779541
12: loss: 27.097719192504883
12: valid loss 19.532703399658203
12: saving model to results
13: loss: 26.579416275024414
14: loss: 21.958843231201172
14: valid loss 23.53536033630371
15: loss: 19.677995681762695
16: loss: 17.906358718872

In [ ]:
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

soundstream.load(f"./{soundstream_ckpt}")

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3,
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    codec = soundstream,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 70
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

training with dataset of 56 samples and validating with randomly splitted 3 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 65.87991333007812
0: valid loss 46.00760269165039
0: saving model to results
1: loss: 53.637046813964844
2: loss: 40.55066680908203
3: loss: 39.247562408447266
4: loss: 36.266170501708984
5: loss: 45.16120910644531
6: loss: 25.04305648803711
7: loss: 14.215118408203125
8: loss: 20.674407958984375
9: loss: 15.495322227478027
10: loss: 17.833988189697266
11: loss: 19.73443603515625
12: loss: 21.898698806762695
13: loss: 16.026182174682617
14: loss: 13.337745666503906
15: loss: 11.153929710388184
16: loss: 14.082469940185547
17: loss: 19.795738220214844
18: loss: 11.861931800842285
19: loss: 14.470977783203125
20: loss: 10.093576431274414
21: loss: 5.557170867919922
22: loss: 13.189838409423828
23: loss: 10.095176696777344
24: loss: 10.730398178100586
25: loss: 10.747623443603516
26: loss: 9.027376174926758
27: loss: 8.18572

In [ ]:
audiolm = AudioLM(
    wav2vec = wav2vec,
    codec = soundstream,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer,
)
generated_wav = audiolm(batch_size = 1)
#generate_wav = audiolm(text = ['cirping of birds'])


generating fine: 100%|██████████| 512/512 [00:34<00:00, 14.79it/s]


In [ ]:

output_path = "out.wav"
sample_rate = 44100
torchaudio.save(output_path, generated_wav.cpu(), sample_rate)